<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/fluxAcrossRuns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set up environment

In [ ]:
!pip install cfgrib

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import cfgrib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#User parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initStart = dt(year=2024, month=10, day=6, hour=12) #Time when Milton began its most rapid intensification
initEnd = dt(year=2024, month=10, day=8, hour=0)

fHourStep = 3       #Normally 3 for HAFS-A
forecastLength = 48 #Normally 126 for HAFS-A.
runStep = 6

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

atmTop = 875
oceFloor = 100

Get ATCF data and find interested point

#Retrieve HAFS-A Data

Find times needed

In [ ]:
fcastTimes = {} #Key: initiation, item: valid time list

initTime = initStart
while initTime <= initEnd:
  validTime = initTime
  fcastTimes[initTime] = []
  fhour = 0

  while fhour <= forecastLength:
    fcastTimes[initTime].append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

  initTime += pd.Timedelta(hours=runStep)

In [ ]:
fcastTimes

Find storm location in HAFS-A from ATCF files. Used to find along-storm profile.

In [ ]:
cols = ["BASIN", "CY", "YYYYMMDDHH", "TECHNUM/MIN", "TECH", "TAU", "LatN/S", "LonE/W",
    "VMAX", "MSLP", "TY", "RAD", "WINDCODE", "RAD1", "RAD2", "RAD3", "RAD4",
    "POUTER", "ROUTER", "RMW", "GUSTS", "EYE", "SUBREGION", "MAXSEAS", "INITIALS",
    "DIR", "SPEED", "STORMNAME", "DEPTH", "SEAS", "SEASCODE", "SEAS1", "SEAS2",
    "SEAS3", "SEAS4", "USERDEFINED1", "Thermo1", "Thermo2", "Thermo3", "Thermo4",
    "Thermo5", "Thermo6", "Thermo7", "USERDEFINED2", "DT", "SHR82", "SHR81_1",
    "SHR82_2",  "USERDEFINED3", "SST", "USERDEFINED4", "ARMW1", "ARMW2"]

In [ ]:
tcLocs = {}
tcInt = {}

for runTime in fcastTimes.keys():

  initStr, initHour = runTime.strftime("%Y%m%d_%H").split("_")
  tcLocs[runTime] = {}
  tcInt[runTime] = {}

  atcfURL = f"https://noaa-nws-hafs-pds.s3.amazonaws.com/hfsa/{initStr}/{initHour}/{tcNum}l.{initStr}{initHour}.hfsa.trak.atcfunix"
  atcfFile = "atcf_" + initStr + "_" + initHour + ".csv"
  !wget -O {atcfFile} {atcfURL}
  atcf = pd.read_csv(atcfFile, names=cols)

  for valid in fcastTimes[runTime]:

    fHour = int((valid-runTime).total_seconds() / 3600)
    pointLat = int(atcf[atcf.TAU==fHour]["LatN/S"].iloc[0].replace("N", ""))/10
    pointLon = int(atcf[atcf.TAU==fHour]["LonE/W"].iloc[0].replace("W", ""))/-10 #Assume western hemisphere
    tcLocs[runTime][valid] = (pointLat, pointLon)
    tcInt[runTime][valid] = int(atcf[atcf.TAU==fHour]["VMAX"].iloc[0])

In [ ]:
atm = {}

for runTime in fcastTimes.keys():

  initStr = runTime.strftime("%Y%m%d%H")
  atm[runTime] = {}

  for valid in fcastTimes[runTime]:

    atm[runTime][valid] = {}

    fhour = str(int((valid-runTime).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour

    atmFile = "hafsa_" + initStr + "_f" + fhour + ".nc"
    atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
    atmData = xr.open_dataset(atmPath)

    point = tcLocs[runTime][valid]
    levels = atmData.isobaricInhPa.data[atmData.isobaricInhPa.data>atmTop] #Get lowest point; should be 1000 hPa

    #Longitude in atm files are in degrees east, but are -180 - 180 in oce files. point has them from -180 - 180
    validPoint = atmData.sel(latitude=point[0], longitude=point[1] + 360, method="nearest")
    for level in levels:
      atm[runTime][valid][level] = {}
      atm[runTime][valid][level]["T"] = float(validPoint.sel(isobaricInhPa=level).t.data)
      atm[runTime][valid][level]["q"] = float(validPoint.sel(isobaricInhPa=level).q.data) * 1000 #Convert from kg/kg to g/kg
      atm[runTime][valid][level]["u"] = float(validPoint.sel(isobaricInhPa=level).u.data)
      atm[runTime][valid][level]["v"] = float(validPoint.sel(isobaricInhPa=level).v.data)
      atm[runTime][valid][level]["gh"] = float(validPoint.sel(isobaricInhPa=level).gh.data)
      atm[runTime][valid][level]["sst"] = float(validPoint.sst.data)
      atm[runTime][valid][level]["shf"] = float(validPoint.ishf.data)
      atm[runTime][valid][level]["lhf"] = float(validPoint.slhtf.data)

In [ ]:
oce = {}
getOce = False

for runTime in fcastTimes.keys():

  if getOce == False: break
  initStr = runTime.strftime("%Y%m%d%H")
  oce[runTime] = {}

  for valid in fcastTimes[runTime]:
    fhour = str(int((valid-runTime).total_seconds() / 3600))
    while len(fhour) < 3: fhour = "0" + fhour
    oceFile = "mom6_" + initStr + "_f" + fhour + ".nc"
    ocePath = dataPath + "mom6Output/" + subfolder + oceFile

    if oceFile == 'mom6_2024100800_f000.nc': #This file is missing
      for layer in oceData.z_l.data:
        oce[initTime][valid][layer] = {"T":np.nan, "s":np.nan, "u":np.nan, "v":np.nan}
      continue

    oceData = xr.open_dataset(ocePath, decode_times=False)

    point = tcLocs[runTime][valid]
    levels = oceData.z_l.data[oceData.z_l.data<=oceFloor] #Retrieves most shallow layer; should be 1 m

    oce[runTime][valid] = {}

    for level in levels:
      oce[runTime][valid][level] = {}
      oce[runTime][valid][level]["T"] = float(oceData.sel(yh=point[0], xh=point[1], z_l=level, method="nearest").temp.data) + 273.15 #This is potential temperature. Converted from C to K.
      oce[runTime][valid][level]["s"] = float(oceData.sel(yh=point[0], xh=point[1], z_l=level, method="nearest").so.data)
      oce[runTime][valid][level]["u"] = float(oceData.sel(yh=point[0], xq=point[1], z_l=level, method="nearest").uo.data)
      oce[runTime][valid][level]["v"] = float(oceData.sel(yq=point[0], xh=point[1], z_l=level, method="nearest").vo.data)
      oce[runTime][valid][level]["sst"] = float(oceData.sel(yh=point[0], xh=point[1], method="nearest").SST.data) + 273.15
      oce[runTime][valid][level]["ssh"] = float(oceData.sel(yh=point[0], xh=point[1], method="nearest").SSH.data)
      oce[runTime][valid][level]["shf"] = float(oceData.sel(yh=point[0], xh=point[1], method="nearest").sensible.data)
      oce[runTime][valid][level]["lhf"] = float(oceData.sel(yh=point[0], xh=point[1], method="nearest").latent.data)

#Figures

Sensible heat flux with intensity

In [ ]:
# Get the number of model runs
num_runs = len(atm)

# Create a figure with a subplot for each run time
fig, axes = plt.subplots(num_runs, 1, figsize=(8, 2 * num_runs), sharex=True)

# Ensure axes is an array even if there's only one run
if num_runs == 1: axes = [axes]

for i, (run_time, valid_times_data) in enumerate(atm.items()):
    times = sorted(valid_times_data.keys())
    shf_values = [valid_times_data[time][1000.0]["shf"] for time in times]
    intensity_values = [tcInt[run_time][time] for time in times] # Get intensity values

    ax1 = axes[i]
    ax2 = ax1.twinx()

    ax1.plot(times, shf_values, label='Sensible Heat Flux', color="darkorange")
    ax2.plot(times, intensity_values, label='Intensity', color="black")

    ax1.set_ylabel("Sensible Heat Flux (W/m$^2$)")
    ax2.set_ylabel("Intensity (kt)")
    ax1.set_title(f'Model Run: {run_time.strftime("%m-%d %HUTC")}')
    ax1.grid(True, alpha=0.5)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

# Set common x-label for the last subplot
axes[-1].set_xlabel("Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Latent heat flux and intensity

In [ ]:
# Get the number of model runs
num_runs = len(atm)

# Create a figure with a subplot for each run time
fig, axes = plt.subplots(num_runs, 1, figsize=(8, 2 * num_runs), sharex=True)

# Ensure axes is an array even if there's only one run
if num_runs == 1: axes = [axes]

for i, (run_time, valid_times_data) in enumerate(atm.items()):
    times = sorted(valid_times_data.keys())
    shf_values = [valid_times_data[time][1000.0]["lhf"] for time in times]
    intensity_values = [tcInt[run_time][time] for time in times] # Get intensity values

    ax1 = axes[i]
    ax2 = ax1.twinx()

    ax1.plot(times, shf_values, label='Latent Heat Flux', color="green")
    ax2.plot(times, intensity_values, label='Intensity', color="black")

    ax1.set_ylabel("Latent Heat Flux (W/m$^2$)")
    ax2.set_ylabel("Intensity (kt)")
    ax1.set_title(f'Model Run: {run_time.strftime("%m-%d %HUTC")}')
    ax1.grid(True, alpha=0.5)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

# Set common x-label for the last subplot
axes[-1].set_xlabel("Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

Sensible + Latnet heat flux and change in intensity

In [ ]:
# Get the number of model runs
num_runs = len(atm)

# Create a figure with a subplot for each run time
fig, axes = plt.subplots(num_runs, 1, figsize=(8, 2 * num_runs), sharex=True)

# Ensure axes is an array even if there's only one run
if num_runs == 1: axes = [axes]

for i, (run_time, valid_times_data) in enumerate(atm.items()):
    times = sorted(valid_times_data.keys())
    validTimes = times[1:]
    pastTimes = times[:-1]

    shf_values = [valid_times_data[time][1000.0]["lhf"]+valid_times_data[time][1000.0]["shf"] for time in validTimes]
    dIntensity = [tcInt[run_time][validTime]-tcInt[run_time][pastTime] for validTime, pastTime in zip(validTimes,pastTimes)] # Get intensity values

    ax1 = axes[i]
    ax2 = ax1.twinx()

    ax1.plot(validTimes, shf_values, label='Latent Heat Flux', color="magenta")
    ax2.plot(validTimes, dIntensity, label='$\Delta$Vmax', color="black")

    ax1.set_ylabel("Latent Heat Flux (W/m$^2$)")
    ax2.set_ylabel("$\Delta$Intensity (kt)")
    ax1.set_title(f'Model Run: {run_time.strftime("%m-%d %HUTC")}')
    ax1.grid(True, alpha=0.5)
    ax1.legend(loc='upper left')
    ax2.legend(loc='upper right')

# Set common x-label for the last subplot
axes[-1].set_xlabel("Time")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()